In [4]:
import pystac_client
import planetary_computer

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [23]:
time_range = "2024-06-01/2024-06-30"
# bbox = [-122.2751, 47.5469, -121.9613, 47.7458]
bbox = [1.283083,43.439552,1.302631,43.451268]
# bbox = Ox (31)

search = catalog.search(collections=["landsat-c2-l2"], bbox=bbox, datetime=time_range)
# items = search.get_all_items()
items = search.item_collection()
len(items)

6

In [18]:
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
            [1.283083,43.439552],
            [1.302631,43.302631],
            [1.302631,43.302631],
            [1.283083,43.302631],
            [1.283083,43.439552],
        ]
    ],
}

time_range = "2024-06-01/2024-06-30"

search = catalog.search(
    collections=["landsat-c2-l2"], intersects=area_of_interest, datetime=time_range
)

In [24]:
len(items)

6

In [20]:
search

In [25]:
import geopandas

df = geopandas.GeoDataFrame.from_features(items.to_dict(), crs="epsg:4326")
df

,geometry,gsd,created,sci:doi,datetime,platform,proj:shape,description,instruments,eo:cloud_cover,...,landsat:scene_id,landsat:wrs_path,landsat:wrs_type,view:sun_azimuth,landsat:correction,view:sun_elevation,landsat:cloud_cover_land,landsat:collection_number,landsat:collection_category,proj:code
0,"POLYGON ((0.93365 44.2381, 0.38038 42.525, 2.5...",30,2024-07-10T09:24:37.723939Z,10.5066/P9OGBGM6,2024-06-24T10:35:30.200884Z,landsat-8,"[7971, 7861]",Landsat Collection 2 Level-2,"[oli, tirs]",11.57,...,LC81980302024176LGN00,198,2,133.795140,L2SP,64.241283,11.57,02,T1,EPSG:32631
1,"POLYGON ((-0.62675 44.23538, -1.17848 42.5222,...",30,2024-06-26T09:09:02.217478Z,10.5066/P9OGBGM6,2024-06-23T10:41:27.418211Z,landsat-9,"[8041, 7941]",Landsat Collection 2 Level-2,"[oli, tirs]",73.86,...,LC91990302024175LGN00,199,2,133.729813,L2SP,64.246992,73.86,02,T1,EPSG:32631
2,"POLYGON ((0.89504 44.23551, 0.34545 42.52177, ...",30,2024-06-19T09:10:07.845224Z,10.5066/P9OGBGM6,2024-06-16T10:35:26.637558Z,landsat-9,"[7951, 7851]",Landsat Collection 2 Level-2,"[oli, tirs]",4.83,...,LC91980302024168LGN00,198,2,134.478554,L2SP,64.418157,4.83,02,T1,EPSG:32631
3,"POLYGON ((-0.60065 44.23863, -1.15535 42.52574...",30,2024-09-13T09:22:13.030127Z,10.5066/P9OGBGM6,2024-06-15T10:41:32.998525Z,landsat-8,"[8051, 7941]",Landsat Collection 2 Level-2,"[oli, tirs]",70.94,...,LC81990302024167LGN00,199,2,134.680119,L2SP,64.436792,70.94,02,T1,EPSG:32631
4,"POLYGON ((0.93776 44.23872, 0.38438 42.52536, ...",30,2024-10-02T09:24:26.677324Z,10.5066/P9OGBGM6,2024-06-08T10:35:19.932569Z,landsat-8,"[7971, 7851]",Landsat Collection 2 Level-2,"[oli, tirs]",55.31,...,LC81980302024160LGN00,198,2,135.842640,L2SP,64.258141,55.31,02,T1,EPSG:32631
5,"POLYGON ((-0.6647 44.23551, -1.21646 42.52216,...",30,2024-06-14T09:14:35.700612Z,10.5066/P9OGBGM6,2024-06-07T10:41:46.419305Z,landsat-9,"[8041, 7941]",Landsat Collection 2 Level-2,"[oli, tirs]",93.06,...,LC91990302024159LGN00,199,2,136.046018,L2SP,64.211212,93.06,02,T1,EPSG:32631


In [26]:
selected_item = min(items, key=lambda item: item.properties["eo:cloud_cover"])
print(selected_item)

<Item id=LC09_L2SP_198030_20240616_02_T1>


In [27]:
import rich.table

table = rich.table.Table("Asset Key", "Description")
for asset_key, asset in selected_item.assets.items():
    table.add_row(asset_key, asset.title)

table

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Asset Key        ┃ Description                                                          ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ qa               │ Surface Temperature Quality Assessment Band                          │
│ ang              │ Angle Coefficients File                                              │
│ red              │ Red Band                                                             │
│ blue             │ Blue Band                                                            │
│ drad             │ Downwelled Radiance Band                                             │
│ emis             │ Emissivity Band                                                      │
│ emsd             │ Emissivity Standard Deviation Band                                   │
│ trad             │ Thermal Radiance Band                                                │
│ urad             │ Upwelled Radiance Band                                               │
│ atran            │ Atmospheric Transmittance Band                                       │
│ cdist            │ Cloud Distance Band                                                  │
│ green            │ Green Band                                                           │
│ nir08            │ Near Infrared Band 0.8                                               │
│ lwir11           │ Surface Temperature Band                                             │
│ swir16           │ Short-wave Infrared Band 1.6                                         │
│ swir22           │ Short-wave Infrared Band 2.2                                         │
│ coastal          │ Coastal/Aerosol Band                                                 │
│ mtl.txt          │ Product Metadata File (txt)                                          │
│ mtl.xml          │ Product Metadata File (xml)                                          │
│ mtl.json         │ Product Metadata File (json)                                         │
│ qa_pixel         │ Pixel Quality Assessment Band                                        │
│ qa_radsat        │ Radiometric Saturation and Terrain Occlusion Quality Assessment Band │
│ qa_aerosol       │ Aerosol Quality Assessment Band                                      │
│ tilejson         │ TileJSON with default rendering                                      │
│ rendered_preview │ Rendered preview                                                     │
└──────────────────┴──────────────────────────────────────────────────────────────────────┘

In [28]:
selected_item.assets["rendered_preview"].to_dict()

{'href': 'https://planetarycomputer.microsoft.com/api/data/v1/item/preview.png?collection=landsat-c2-l2&item=LC09_L2SP_198030_20240616_02_T1&assets=red&assets=green&assets=blue&color_formula=gamma+RGB+2.7%2C+saturation+1.5%2C+sigmoidal+RGB+15+0.55&format=png',
 'type': 'image/png',
 'title': 'Rendered preview',
 'rel': 'preview',
 'roles': ['overview']}

In [29]:
from IPython.display import Image

Image(url=selected_item.assets["rendered_preview"].href, width=500)

In [30]:
selected_item.assets["blue"].href[:250]

'https://landsateuwest.blob.core.windows.net/landsat-c2/level-2/standard/oli-tirs/2024/198/030/LC09_L2SP_198030_20240616_20240617_02_T1/LC09_L2SP_198030_20240616_20240617_02_T1_SR_B2.TIF?st=2025-03-30T14%3A26%3A26Z&se=2025-03-31T15%3A11%3A26Z&sp=rl&sv'

In [31]:
import requests

requests.head(selected_item.assets["blue"].href).status_code

200